Text Generation with Markov Chains in Python
============================================

In [1]:
files = ['text/grimm_tales.txt']

text = ""
for f in files:
    with open(f, 'r') as f:
        text += f.read()
print(text[:500])

THE GOLDEN BIRD

A certain king had a beautiful garden, and in the garden stood a tree
which bore golden apples. These apples were always counted, and about
the time when they began to grow ripe it was found that every night one
of them was gone. The king became very angry at this, and ordered the
gardener to keep watch all night under the tree. The gardener set his
eldest son to watch; but about twelve o’clock he fell asleep, and in
the morning another of the apples was missing. Then the second


Weather as a Markov Chain
-------------------------

![alt text](images/markov_weather.png "Weather")

Matrix representation (rows are current state, columns are next state):

| | Sunny | Cloudy | Rainy |
| --- | --- | --- | --- |
| **Sunny** | 0.6 | 0.1 | 0.3 |
| **Cloudy** | 0.3 | 0.3 | 0.4 |
| **Rainy** | 0.3 | 0.2 | 0.5 |


Text as a Markov Chain
----------------------

**The cat ran over the dog.**

![alt text](images/markov_text1.png "Text")

Matrix representation (rows are current state, columns are next state):

| | the | cat | ran | over | dog | . |
| --- | --- | --- | --- | --- | --- | --- |
| **the** | 0 | 0.5 | 0 | 0 | 0.5 | 0 |
| **cat** | 0 | 0 | 1 | 0 | 0 | 0 |
| **ran** | 0 | 0 | 0 | 1 | 0 | 0 |
| **over** | 1 | 0 | 0 | 0 | 0 | 0 |
| **dog** | 0 | 0 | 0 | 0 | 0 | 1 |
| **.** | 0 | 0 | 0 | 0 | 0 | 1 |



Define states as the distinct word tokens

In [2]:
import re
text = re.sub("[^A-z,.!?'\n ]+", "", text)
text = re.sub("([,.!?])", r" \1 ", text)

tokens = text.lower().split()
distinct_states = list(set(tokens))

Define transition matrix

In [3]:
from scipy.sparse import csr_matrix
m = csr_matrix(
    (len(distinct_states), len(distinct_states)),
    dtype = int
)

state_index = dict(
    [(state, idx_num) for idx_num, state in enumerate(distinct_states)]
)

Count transitions and fill in transition matrix

In [4]:
for i in range(len(tokens)-1):
    row = state_index[tokens[i]]
    col = state_index[tokens[i+1]]
    m[row,col] += 1

/home/studio-lab-user/.conda/envs/ds309/lib/python3.9/site-packages/scipy/sparse/_index.py:103: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  self._set_intXint(row, col, x.flat[0])


Generate new text

In [5]:
import numpy as np

start_state_index = np.random.randint(len(distinct_states))
state = distinct_states[start_state_index]
num_sentences = 0
output = state.capitalize()
capitalize = False


while num_sentences < 3: 
    
    row = m[state_index[state], :]
    probabilites = row / row.sum()
    probabilites = probabilites.toarray()[0]
    
    next_state_index = np.random.choice(
        len(distinct_states), 
        1,
        p = probabilites
    )
    next_state = distinct_states[next_state_index[0]]
    
    if next_state in ('.', '!', '?'):
        output += next_state + '\n\n'
        capitalize = True
        num_sentences += 1
    elif next_state == ",":
        output += next_state
    else:
        if capitalize:
            output += next_state.capitalize()
            capitalize = False
        else:
            output += " " + next_state
    
    state = next_state
    
print(output)

Art the cat climbed up here is not see who shall die, and then they will kill the young man can imagine that the cook my stomach, with a pig by the fish darted straight on the forest, said he arose she felt even though he still upon their child.

And benches, now to herself home to no doubt at last rays of four brothers, half so long sought you have been to your pipe and then he willingly take it was that the garden of land, wishing to himself across the old woman was so sorrowful but when the gold chain in the king has a little tailor he rode.

The blue sky, and the time in, set out together how he took off the others went into but when she did i would sell?




k-Word Markov Chain
-------------------

**The cat ran over the dog.**

![alt text](images/markov_text2.png "Text")

Matrix representation (rows are current state, columns are next state):

| | the cat | cat ran | ran over | over the | the dog | dog. |
| --- | --- | --- | --- | --- | --- | --- |
| **the cat**  | 0 | 1 | 0 | 0 | 0 | 0 |
| **cat ran**  | 0 | 0 | 1 | 0 | 0 | 0 |
| **ran over** | 0 | 0 | 0 | 1 | 0 | 0 |
| **over the** | 0 | 0 | 0 | 0 | 1 | 0 |
| **the dog**  | 0 | 0 | 0 | 0 | 0 | 1 |
| **dog.**     | 0 | 0 | 0 | 0 | 0 | 1 |




Define states as consecutive token pairs

In [6]:
k = 3
tokens = text.lower().split()
states = [tuple(tokens[i:i+k]) for i in range(len(tokens)-k+1)]
distinct_states = list(set(states))

Define and fill transition matrix

In [ ]:
from scipy.sparse import csr_matrix
m = csr_matrix(
    (len(distinct_states), len(distinct_states)),
    dtype = int
)

state_index = dict(
    [(state, idx_num) for idx_num, state in enumerate(distinct_states)]
)

for i in range(len(tokens)-k):
    state = tuple(tokens[i:i+k])
    next_state = tuple(tokens[i+1:i+1+k])
    row = state_index[state]
    col = state_index[next_state]
    m[row,col] += 1

Generate new text

In [ ]:
import numpy as np

start_state_index = np.random.randint(len(distinct_states))
state = distinct_states[start_state_index]
num_sentences = 0
output = ' '.join(state).capitalize()
capitalize = False


while num_sentences < 5: 
    
    row = m[state_index[state], :]
    probabilites = row / row.sum()
    probabilites = probabilites.toarray()[0]
    
    next_state_index = np.random.choice(
        len(distinct_states), 
        1,
        p = probabilites
    )
    next_state = distinct_states[next_state_index[0]]
    
    if next_state[-1] in ('.', '!', '?'):
        output += next_state[-1] + '\n\n'
        capitalize = True
        num_sentences += 1
    elif next_state[-1] == ",":
        output += next_state[-1]
    else:
        if capitalize:
            output += next_state[-1].capitalize()
            capitalize = False
        else:
            output += " " + next_state[-1]
    
    state = next_state
    
print(output)